In [33]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy.stats import norm
from scipy.optimize import fsolve
from scipy.optimize import brentq

In [2]:
# Price for zero-coupon bond with stochastic interest rate under Vasicek's model
def ZC_Vasicek(F, r, kappa, theta, sigma, t, T):
    
    delta_T = T - t
    
    B = (1 - np.exp(-kappa * delta_T)) / kappa
    
    A = np.exp((theta - (sigma**2) / (2 * kappa**2)) * (B - delta_T) - (sigma**2 / (4 * kappa)) * B**2)
    
    bond_price = F * A * np.exp(-B * r)
    
    return bond_price

In [5]:
print(ZC_Vasicek(F=100, r=0.02, kappa = 0.1, theta = 0.03, sigma=0.02, t=0, T=1))

97.97852638018804


In [7]:
def rpoi(a, y, lam, t, T):
    return np.exp(lam * (T-t) * (np.exp(a) - 1) - a * y)

def rlog(a, y, mu, sig, n):
    # Calculate the exponents
    exponent1 = ((mu + a)**2 - mu**2) / (2 * sig**2)
    exponent2 = a * y / (sig**2)
    
    # Compute the final result using logarithms to avoid overflow
    log_result = (exponent1 * n) - exponent2  # Logarithmic equivalent of the division
    
    # Calculate the result using np.exp
    result = np.exp(log_result)
    
    return result

def g(b=0.07,lam=35,D=9000000000,T=1, mu=18.4, sig=1):
    lhs = 2 * lam * T * b / sig**2 * np.exp(b**2 / (2 * sig**2))
    z = (np.log(D) - mu + b) / sig
    rhs = norm.pdf(z) / (sig * norm.sf(z))
    return lhs - rhs

In [9]:
# MC with lognormal distribution for the size of losses
def naive_MC_log(nr, lam, D, mu, sig, T):
    h = []
    poissons = np.random.poisson(lam=lam*T, size=nr)
    for i in range(nr):
        x = np.sum(np.random.lognormal(mean=mu, sigma=sig, size = poissons[i]))
        h.append(int(x>D))
    return np.cumsum(h)/np.arange(1,nr+1)

In [35]:
# Importance sampling for default probability
def MC_IS_log_poi(nr=5000, lam=35,D=9000000000, mu=18.4, sig=1, t=0, T=1):
   
    def root_func(b):
        return g(b, lam, D, T, mu, sig)
        
    # Initial variables
    #a_log = fsolve(root_func, x0=0.05)[0]
    a_log = brentq(g, 0.001, 0.2)
    a_poi = a_log**2/(2*sig**2)
    poisson_means = lam * T * np.exp(a_poi)
    new_mu = mu+a_log

    # Preallocate memory for cumulative sums
    h = np.zeros(nr)
    r_poi = np.zeros(nr)
    r_log = np.zeros(nr)
    
    # Loop over the number of simulations
    for i in range(nr):
        # Generate Poisson-distributed count
        poissons = np.random.poisson(lam=poisson_means)

        # Generate the exponential random variables for this count
        # Directly sum them without creating large intermediate arrays
        x = np.random.normal(loc=new_mu, scale=sig, size=poissons)
        y = np.sum(x)
        z = np.sum(np.exp(x))

        # Compute h and r for this simulation
        h[i] = int(z > D)
        r_poi[i] = rpoi(a_poi, poissons, lam, t, T)
        r_log[i] = rlog(a_log, y, mu, sig, poissons)

    # Calculate cumulative sum and return the average at each step
    cumulative_sum = np.cumsum(h * r_poi * r_log)
    cumulative_avg = cumulative_sum / np.arange(1, nr + 1)

    return cumulative_avg

In [37]:
# Zero-coupon CAT bond pricing
def CAT_ZC_Vasicek(F, r, kappa, theta, sigma, D, lam, mu, sig, nr, t, T):
    if lam * T * np.exp(mu + sig**2 / 2) < D:
        price = ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_log_poi(nr, lam, D, mu, sig, t, T)[-1])
    else:  # Case: N == 0 and lam * T * k * th > D
        price = ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-naive_MC_log(nr, lam, D, mu, sig, T)[-1])
    return price

In [39]:
print(CAT_ZC_Vasicek(F=100, r=0.02, kappa=0.1, theta=0.03, sigma=0.02, lam=35, D=9000000000, mu=18.4, sig=1, nr=5000, t=0, T=1))

94.9286478733177


In [41]:
# CAT bond with coupons pricing
def CAT_C_Vasicek(F, c, r, kappa, theta, sigma, D, lam, mu, sig, nr, t, N, T):
    c_sum = 0
    dt = T/N
    
    for coupon_count in range (N):
        if lam * (coupon_count+1) * dt * np.exp(mu + sig**2 / 2) < D:
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-MC_IS_log_poi(nr, lam, D, mu, sig, t, T=(coupon_count+1)*dt)[-1])
        else:  # Case: N == 0 and lam * T * k * th > D
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-naive_MC_log(nr, lam, D, mu, sig, T=(coupon_count+1)*dt)[-1])
    
    if lam * T * np.exp(mu + sig**2 / 2) < D:
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_log_poi(nr, lam, D, mu, sig, t, T)[-1])
    else:  # Case: N == 0 and lam * T * k * th > D
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-naive_MC_log(nr, lam, D, mu, sig, T)[-1])
    
    return c_sum

In [43]:
print(CAT_C_Vasicek(F=100, c=0.05, r=0.02, kappa=0.1, theta=0.03, sigma=0.02, lam=35,D=9000000000, mu=18.4, sig=1, nr=5000, t=0, N=3, T=1))

109.3034976483269


In [45]:
# Parameter ranges
F = 1000  # Fixed face value
t = 0     # Fixed initial time
#lam = 1   # Fixed Poisson rate
mu = 18.4    # Fixed lognormal shape
sig = 1    # Fixed lognormal scale
nr = 5000
c=0.05
#r=0.03
kappa=0.2
theta=0.03
sigma=0.02



# lam_values = np.linspace(30, 40, 50)       # Range for lambda
# D_values = np.linspace(7, 11, 21)*1000000000           # Range for threshold
# N_values = [2, 4, 6, 12]      # Range for coupon frequency
# T_values = np.linspace(90, 720, 43)*1/360

In [47]:
from tqdm import tqdm
import csv

N_sim = 600000
N_values = [0, 2, 3, 4, 6, 8, 10, 12]
chunk_size = 10000  
start_chunk = 1  # Allows skipping initial chunks

# Compute total chunks
total_chunks = (N_sim) // chunk_size
results = []

# CSV file path
csv_file = 'CAT_price_log.csv'

# Prepare CSV file header if needed
csv_header = ["c", "r", "kappa", "theta", "sigma", "lambda", "D", "N", "T", "Price"]

# Open CSV file in write mode to create the header
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header)  # Write the header to the CSV file
    
    print(f"Running {N_sim} simulations in {total_chunks} chunks...\n")

    for chunk_index in range(1, total_chunks + 1):
        if chunk_index < start_chunk:
            continue  # Skip initial chunks if needed

        chunk_results = []

        for _ in tqdm(range(chunk_size), desc=f"Processing chunk {chunk_index}/{total_chunks}"):
            N = np.random.choice(N_values)  # Randomly select N
            r = np.random.uniform(0, 0.08)
            lam = np.random.uniform(30, 40)
            D = np.random.uniform(7, 13)*1000000000 
            T = np.random.uniform(90, 720)*1/360

            # Condition to determine which function to use
            if N == 0:
                price = CAT_ZC_Vasicek(F, r, kappa, theta, sigma, D, lam, mu, sig, nr, t, T)
            else:
                price = CAT_C_Vasicek(F, c, r, kappa, theta, sigma, D, lam, mu, sig, nr, t, N, T)

            chunk_results.append({
                "c": c,
                "r": r,
                "kappa": kappa,
                "theta": theta,
                "sigma": sigma,
                "lambda": lam,
                "D": D,
                "N": N,
                "T": T,
                "Price": price
            })

        # Save the chunk results into the CSV file after each chunk finishes
        for result in chunk_results:
            writer.writerow(result.values())

        results.extend(chunk_results)  # Save chunk results


print("Simulation complete. Results saved to CSV file.")    

Running 600000 simulations in 60 chunks...



Processing chunk 60/60: 100%|█████████████| 10000/10000 [30:14<00:00,  5.51it/s]

Simulation complete. Results saved to CSV file.
